<a href="https://colab.research.google.com/github/jordanfaroz/AI-and-Tf-projects/blob/main/new_chatbot_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot using tensorflow, NLP and python

## Step 1: Set up the environment
-installing tensorflow and importing required libraries

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
import random
import json

## Step 2: Prepare the data

Creating a JSON file with your training data. It should have a list of patterns and responses.

In [ ]:
{
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hello", "Hey"],
      "responses": ["Hello!", "Hi there!", "Hey! How can I help you?"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Bye", "See you later", "Goodbye"],
      "responses": ["Goodbye!", "See you later!", "Take care!"]
    }
  ]
}

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
with open('data.json', 'r') as file:
    contents = file.read()
    print(contents)

## Loading and preprocessing the data in Python:

In [ ]:
with open('data.json', 'r') as file:
    data = json.load(file)

# Extract patterns and responses from the loaded data
patterns = []
responses = []
tags = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        responses.append(intent['responses'][random.randint(0, len(intent['responses']) - 1)])
        tags.append(intent['tag'])

## Step 3: Building and training the model

Preprocess the text data:

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the tags to numerical values
label_encoder = LabelEncoder()
encoded_tags = label_encoder.fit_transform(tags)

# Convert the encoded tags to one-hot encoded labels
labels = tf.keras.utils.to_categorical(encoded_tags)

## Define the model architecture:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10, 16, input_length=20),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(labels[0]), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Step 4: Test the chatbot

Preprocess the user's input:

In [ ]:
def preprocess_input(input_text):
    input_seq = tf.keras.preprocessing.text.Tokenizer.texts_to_sequences(input_text)
    padded_input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=20, padding='post')
    return padded_input_seq

## Generate a response from the chatbot

In [ ]:
def get_response(input_text):
    input_seq = preprocess_input(input_text)
    predictions = model.predict(input_seq)
    tag = tags[np.argmax(predictions)]
    responses = []
    for intent in data['intents']:
        if intent['tag'] == tag:
            responses = intent['responses']
    return random.choice(responses)

## Test the chatbot:

In [ ]:
user_input = input("You: ")
response = get_response(user_input)
print("Bot:", response)